# Current site tickets - ENSWEB

TODO
- Remove token
- Improve views
- Improve diff tracking
- hide code

In [1]:
!pip install pandas
!pip install ../personalissues/

Processing /Users/jon/Programming/tools/personalissues
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for personalissues: filename=personalissues-0.1.0-py3-none-any.whl size=4962 sha256=73566b4b9db6896dfd40927d9feb3720e1f5dce3e17a253c237237b31dbfcb92
  Stored in directory: /private/var/folders/0n/f1bqpff95nx876b07rhsfwsm0000gp/T/pip-ephem-wheel-cache-qfg6d0_f/wheels/9a/59/b9/5cf8b4993e79e42ea7bc0141f4b243d54028cf6c48420bade6
Successfully built personalissues
  Attempting uninstall: personalissues
    Found existing installation: personalissues 0.1.0
    Uninstalling personalissues-0.1.0:
      Successfully uninstalled personalissues-0.1.0


In [13]:

import json
import configparser

import pandas as pd
from datetime import datetime


from personalissues.personal_issues import personal_issues

config = configparser.ConfigParser()
config.read('settings.ini')


['settings.ini']

## Update data

In [14]:
DATA_FILE = "frontend-ensweb"
ISSUES_FILE = f"{DATA_FILE}.json"
SUMMARY_FILE = f"{DATA_FILE}_summary.json"

personal_issues(
    "https://www.ebi.ac.uk/panda/jira",
    config['jira']['token'],
    ["imran","andrey","jyo", "jon"],
    "ENSWEB",
    f"frontend-ensweb",
    True
)

Personal issues
Connecting to https://www.ebi.ac.uk/panda/jira
Query: assignee = imran AND status not in (Done, Closed, Resolved, Cancelled) order by updated DESC
Query: assignee = andrey AND status not in (Done, Closed, Resolved, Cancelled) order by updated DESC
Query: assignee = jyo AND status not in (Done, Closed, Resolved, Cancelled) order by updated DESC
Query: assignee = jon AND status not in (Done, Closed, Resolved, Cancelled) order by updated DESC
issues
Getting unassigned and filtering by project ENSWEB
Query: project = ENSWEB AND assignee  is EMPTY  AND status not in (Done, Closed, Resolved, Cancelled) order by updated DESC
Found imran
Found andrey
Found jyo
Found jon
Found unassigned
Done :D


## Load data

In [15]:
with open(ISSUES_FILE) as issue_file:
    issues = json.load(issue_file)

print(f"{len(issues)} issues found")

with open(SUMMARY_FILE) as summary_file:
    summary = json.load(summary_file)

print(f"Summary data found for {len(summary)} people")
    

39 issues found
Summary data found for 5 people


## Ensweb active issues

In [16]:
ignore = ['sum_life_times','avg_life_times']


counts = {}
people_counts = {}

view = []

for person, data in summary.items():
    person_count = {key:value for key,value in data['issues'].items()}
    
    for key,value in person_count.items():
        if key not in counts:
            counts[key] = 0
            
        if person != 'unassigned':
            counts[key] += value
        
    people_counts[person] = person_count


fields = [key for key,_ in counts.items()]

for person, data in people_counts.items():
    flattened = [person]
    flattened.extend([data.get(key,0) for key in fields])
    view.append(flattened)
    
flattened = ["everyone"]
flattened.extend([value for _, value in counts.items()])
view.append(flattened)

columns = ["Who"]
columns.extend(fields)

df = pd.DataFrame(view, columns = columns)
df.style

        

,Who,total,In Progress,Reported,In Review
0,imran,4,2,2,0
1,andrey,6,0,6,0
2,jyo,5,1,4,0
3,jon,6,0,4,2
4,unassigned,18,0,18,0
5,everyone,21,3,16,2


## Issue details

In [17]:
columns = ['issue','summary','person','status','life']

LINK_FORMAT = "<a target=\"new\" href=\"https://www.ebi.ac.uk/panda/jira/browse/{id}\">{id}</a>"

view = []
for issue in issues:
    view.append([issue[key] for key in columns])
    
for row in view:
    row[0] = LINK_FORMAT.format(id=row[0])

df = pd.DataFrame(view, columns=columns)
df.style.bar(subset=['life'], color='#d65f5f')


,issue,summary,person,status,life
0,ENSWEB-6518,Nodes in U6 gene tree move about when collapsed on www staging site,imran,In Progress,467
1,ENSWEB-6820,Broken page in human example self-alignment synteny - 110 vertebrates staging,imran,Reported,50
2,ENSWEB-6824,Link not working on 110/pan compara,imran,Reported,48
3,ENSWEB-6791,Selecting chicken breed breaks taxon selector in Variant Recoder,imran,In Progress,79
4,ENSWEB-5667,3D protein viewer not working with all PDB entries,andrey,Reported,1094
5,ENSWEB-6359,PDBe widget not displaying models even though there are models available,andrey,Reported,616
6,ENSWEB-6522,Pig breeds and Murinae strains displayed in Mammals and Amniotes species trees,andrey,Reported,464
7,ENSWEB-6513,Javascript filtering broken on Orthologue table message box,andrey,Reported,469
8,ENSWEB-6345,License missing from https://github.com/Ensembl/ensemblgenomes.org,andrey,Reported,640
9,ENSWEB-5874,Searching from ensemblgenomes.org gives different results to searching from taxa-specific pages,andrey,Reported,1679
